In [ ]:
library(treeio)
library(ggtree)
library(ggplot2)
library(ape)
library(cowplot)
library(circlize)
library(ggtreeExtra)
library(ggnewscale)

metadata <- read.table('/syn1/liangzhen/jinhua_jilab_project/result/scRNA/cellranger/metadata.T1_T2_T3a.harmony.final.refine.csv',sep=',',header=T,row.names=1)
rownames(metadata) <- metadata$cellName


files <- list.files('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/trees/merge_all_experiments/T1T2T3/')
files <- grep('treefile$',files,value = T)
new_lineageGrp <- read.table('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/new_lineageGrp.csv',sep=',',row.names = 2,header = T)
rownames(new_lineageGrp) <- paste0('C',rownames(new_lineageGrp))
new_lineageGrp$new_lineageGrp <- paste0('C',new_lineageGrp$new_lineageGrp)


for (file in files){
    clone_size <- as.numeric(gsub('.phy.treefile','',unlist(strsplit(file,split = '-'))[3]))
    if (clone_size < 0){
        next
    }
    tree <- read.tree(paste0('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/trees/merge_all_experiments/T1T2T3/',file))
    tree <- root(tree, outgroup = "synthetic", resolve.root = TRUE)
    tree <- drop.tip(tree,tree$tip.label[!(tree$tip.label %in% metadata$cellName)])

    tree_plot <- ggtree(tree,layout = 'fan', open.angle = 0,branch.length='none',size=0.3)

    tree_plot$data$Time <- 'Root'
    tree_plot$data[tree_plot$data$isTip==TRUE & tree_plot$data$label!='synthetic','Time'] <- metadata[tree_plot$data$label[tree_plot$data$isTip==TRUE & tree_plot$data$label!='synthetic'],'time']

    tree_plot$data$cluster <- 'Root'
    tree_plot$data[tree_plot$data$isTip==TRUE & tree_plot$data$label!='synthetic','cluster'] <- as.character(metadata[tree_plot$data$label[tree_plot$data$isTip==TRUE & tree_plot$data$label!='synthetic'],'seurat_clusters_rename'])


    options(repr.plot.width=6, repr.plot.height=6)
    plot <- tree_plot +
        new_scale_fill() +
         geom_fruit(
             geom=geom_tile,
             mapping=aes(fill=Time),
             pwidth=max(tree_plot$data$x)*0.20,
             offset=0.13
         ) + scale_fill_manual(
             name="Time",
             breaks=c('T1','T2','T3','T4'),
             values=c('#6699CC','#B1D66B','#F1A878','#e7298a'),
             guide=guide_legend(keywidth=1, keyheight=1, ncol=1)
         ) + 
         new_scale_fill() +
         geom_fruit(
             geom=geom_tile,
             mapping=aes(fill=cluster),
             pwidth=max(tree_plot$data$x)*0.20,
             offset=0.22
         ) + scale_fill_manual(
             name="cluster",
             breaks=c('1','2','3','4','5'),
             values=c('#a6cee3','#54278f','#e7298a','#1f77b4','#ff6600'),
             guide=guide_legend(keywidth=1, keyheight=1, ncol=1)
         )+ theme(legend.position="none")  
    
    lineageGrp <- unlist(strsplit(file,split = '-'))[2]
    file <- gsub(lineageGrp,new_lineageGrp[lineageGrp,'new_lineageGrp'],file)
    ggsave(plot,file=paste0('/syn1/liangzhen/jinhua_jilab_project/result/Figures/Figure2/trees/',file,'.pdf'),height=log(clone_size,2),width=log(clone_size,2))
    
}
